In [36]:
%matplotlib inline

import os 
import sys 
import time
import hashlib

from datetime  import datetime 

import numpy as np  # learn 
import pandas as pd # learn
from pandas import *
from numpy import *

from scipy import stats # look at scipy
from scipy import linalg
from scipy import *

 
import matplotlib as mplt # learn matplolib 
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

import seaborn as sns 
sns.set_style("whitegrid")
sns.set(rc={"figure.figsize": (14, 6)})

import IPython
from IPython.display import display


import matplotlib
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt
 
import sklearn as sk
import itertools


from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

from pyspark.mllib.regression import LabeledPoint
import random
import time
import copy
import heapq
# from SyntheticDataGenerator import *

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel


from pyspark.mllib.util import MLUtils

In [17]:
# parameters
feature_num=8


In [20]:
raw_rdd = sc.textFile("hdfs://computer61.ant-net/user/tguo/airline_data_1year.csv")
raw_rdd.cache()

print raw_rdd.first()
dta_rdd = raw_rdd.map(lambda line: line.split(",")).map(lambda r:
                                                                (float(r[0]),
                                                                 int(float(r[1])),int(float(r[2])),
                                                                 int(float(r[3])),int(float(r[4])),
                                                                 int(float(r[5])),int(float(r[6])),
                                                                 int(float(r[7])),int(float(r[8]))))

print dta_rdd.first()
print dta_rdd.count()

23.0,0,0,20,2,10,154,11,150
(23.0, 0, 0, 20, 2, 10, 154, 11, 150)
1288326


In [21]:
cate_map={}
for i in range(0, feature_num):
    cate_map.update({i: dta_rdd.map(lambda line: line[ i+1  ]).distinct().count()}  ) 

print cate_map

{0: 1, 1: 3, 2: 31, 3: 7, 4: 14, 5: 2161, 6: 237, 7: 237}


In [31]:
# prepare data

labeled_dta=  dta_rdd.map(lambda line: LabeledPoint(line[0],line[1:feature_num+1]) )
print labeled_dta.first()
# Split the data into training and test sets (30% held out for testing)
(train_dta, test_dta) = labeled_dta.randomSplit([0.8, 0.2])

train_dta.cache() 
test_dta.cache()
# print train_dta.count()
# print test_dta.count()

tmp_rdd = train_dta.map( lambda line: line.label)
ymax= tmp_rdd.max()
ymin= tmp_rdd.min()
print ymax, ymin

(23.0,[0.0,0.0,20.0,2.0,10.0,154.0,11.0,150.0])
1033.0 -1302.0


In [ ]:
#  prepare data
(train_dta, tmp_dta) = labeled_dta.randomSplit([0.6, 0.4])
(test_dta, valid_dta) = tmp_dta.randomSplit([0.75, 0.25])

train_dta.cache() 
test_dta.cache()
valid_dta.cache()

tmp_rdd = train_dta.map( lambda line: line.label)
ymax= tmp_rdd.max()
ymin= tmp_rdd.min()
print ymax, ymin

In [40]:
# single-round test

regTree = DecisionTree.trainRegressor(train_dta, categoricalFeaturesInfo = cate_map,
                                    impurity='variance', maxDepth=8, maxBins=2200)

rf = RandomForest.trainRegressor(train_dta, categoricalFeaturesInfo=cate_map,
                                    numTrees=3, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=4, maxBins=2200)

gbt = GradientBoostedTrees.trainRegressor(train_dta,
                                            categoricalFeaturesInfo=cate_map, numIterations=3,maxBins=2200)

    

In [ ]:
# regressin tree
for dep in range(1:10):
    regTree = DecisionTree.trainRegressor(train_dta, categoricalFeaturesInfo = cate_map,
                                    impurity='variance', maxDepth=dep, maxBins=2200)
    print precision_test(regTree, test_dta)

    

In [ ]:
# random forest 
for tnum in range(2:10):
    rf = RandomForest.trainRegressor(train_dta, categoricalFeaturesInfo=cate_map,
                                    numTrees=tnum, featureSubsetStrategy="auto",
                                    impurity='variance', maxDepth=4, maxBins=2200)
    print precision_test(rf, test_dta)


In [ ]:
#  boosted
for iter_num in range(2:10):
    gbt = GradientBoostedTrees.trainRegressor(train_dta,
                                            categoricalFeaturesInfo=cate_map, numIterations=iter_num,maxBins=2200)
    print precision_test(gbt, test_dta)


In [44]:
# Evaluate model on test instances and compute Normalized root-mean-square deviation (NRMSE)

def precision_test(model, testDta):
    pred = model.predict(testDta.map(lambda x: x.features))
    local_pred = pred.collect()

    labels = testDta.map(lambda x: x.label)
    local_labels = labels.collect()

#     local_predAndLabel= zip(local_pred,local_labels)
    local_resid=map( lambda x,y: (x-y)*(x-y), local_pred,local_labels )
    return sqrt(sum(local_resid)/len(local_pred))/(ymax-ymin)

print precision_test(regTree, test_dta)
print precision_test(rf, test_dta)
print precision_test(gbt, test_dta)


0.0108807595387
0.0110392868024
0.0110391531086
